In [1]:
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error
from xgboost import plot_importance
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
challenge_set_df = pd.read_csv('../../data/encoded_challenge_set.csv')
display(challenge_set_df)

,flight_id,month-day,month,day_in_month,adep,country_code_adep,ades,country_code_ades,aircraft_type,wtc,...,altitude_mean,groundspeed_mean,track_mean,vertical_rate_mean,track_unwrapped_mean,u_component_of_wind_mean,v_component_of_wind_mean,temperature_mean,specific_humidity_mean,tow
0,248763780,0,1,1,69,37,79,34,4,1,...,20996.873271,317.161594,266.904445,12.768124,266.904445,31.354330,28.956784,244.875034,0.001527,54748.000000
1,248760618,0,1,1,216,33,165,78,18,0,...,30316.293173,425.555502,253.416154,66.719357,254.283623,21.483643,-8.009300,227.431051,0.000523,185441.000000
2,248753824,0,1,1,137,88,168,78,7,0,...,28087.460730,346.017174,234.214404,-35.945267,226.498682,28.988018,5.062526,225.055003,0.000467,230396.000000
3,248753852,0,1,1,338,19,169,78,21,0,...,31606.355298,405.077422,260.254022,80.900909,177.655929,32.200133,2.404330,227.891101,0.000931,157615.000000
4,248755934,0,1,1,92,44,58,27,1,1,...,14895.710572,354.972270,131.665419,2754.662045,131.665419,22.697229,20.421709,255.205584,0.001545,70318.447226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369008,258058361,364,12,31,253,36,165,78,21,0,...,27520.238596,352.816281,258.075719,105.885193,258.075719,41.300959,34.816118,230.600595,0.001169,163438.000000
369009,258071247,364,12,31,385,96,15,19,1,1,...,31307.635402,358.665526,312.942321,-74.184695,312.942321,16.449682,-17.756664,222.159498,0.000440,78707.000000
369010,258059152,364,12,31,30,26,80,34,4,1,...,27064.704323,306.001767,278.353987,-246.922841,279.965484,48.607200,26.609141,229.831493,0.001106,62942.750000
369011,258072276,364,12,31,253,36,80,34,1,1,...,29506.571457,302.045249,299.062133,-271.516294,299.062133,45.752148,27.416125,225.945456,0.000713,72611.161024


In [3]:
features = ['month-day', 'month', 'day_in_month', 'adep', 'country_code_adep', 'ades', 'country_code_ades', 
            'aircraft_type', 'wtc', 'airline', 'flight_duration', 'taxiout_time', 'flown_distance', 'actual_offblock_time_hour', 
            'actual_offblock_time_minute', 'actual_offblock_time_hour_minute', 'arrival_time_hour', 'arrival_time_minute', 
            'arrival_time_hour_minute', 'tow']

corr_scores = {'feature': [], 'correlation_score': []}

for feature_1 in features:
    correlation_score = -1
    for feature_2 in features:
        corr_val = np.abs(challenge_set_df[feature_1].corr(challenge_set_df[feature_2]))
        correlation_score += corr_val
    corr_scores['feature'].append(feature_1)
    corr_scores['correlation_score'].append(correlation_score)
    
corr_scores_df = pd.DataFrame.from_dict(corr_scores).sort_values('correlation_score', ascending=False).reset_index(drop=True)

display(corr_scores_df)

,feature,correlation_score
0,tow,4.100006
1,flown_distance,4.034442
2,flight_duration,3.997276
3,wtc,3.905946
4,arrival_time_hour,2.833942
5,arrival_time_hour_minute,2.823801
6,actual_offblock_time_hour,2.652844
7,actual_offblock_time_hour_minute,2.630482
8,country_code_adep,2.360298
9,country_code_ades,2.036341


In [4]:
def generate_combinations(lst):
    all_combinations = []
    for r in range(len(lst) + 1):
        combinations = itertools.combinations(lst, r)
        all_combinations.extend(combinations)
    return all_combinations
    
def rmse(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))
    
def try_combination(features_combination):
    features = challenge_set_df[features_combination]
    target = challenge_set_df['tow']
    X_array = features.values
    y_array = target.values

    mse_scores = []

    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    
    for train_index, test_index in kf.split(X_array):
        X_train, X_test = features.iloc[train_index], features.iloc[test_index]
        y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
        model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse')

        # Train the model
        model.fit(X_train, y_train)
    
        # Predict
        y_pred = model.predict(X_test)
    
        mse_scores.append(rmse(y_test, y_pred))
    
    return np.mean(mse_scores)

In [ ]:
import itertools

features_names = corr_scores_df['feature'].values.tolist()[1:11]
print(features_names)

combinations = generate_combinations(features_names)

# Number of combinations
num_combinations = len(combinations)
print(f"Total number of combinations: {num_combinations}")

models_scores = {'model_number': [], 'features_combination': [], 'rmse_score': []}
model_number = 1
for combo in combinations[1:]:
    combination = list(combo)
    #print(f"{model_number = }")
    #print(f"{combination = }")
    rmse_score = try_combination(combination)
    #print(f"{rmse_score = }")
    models_scores['model_number'].append(model_number)
    models_scores['features_combination'].append(combination)
    models_scores['rmse_score'].append(rmse_score)
    model_number += 1
    
models_scores_df = pd.DataFrame.from_dict(models_scores).sort_values('rmse_score', ascending=True).reset_index(drop=True)
display(models_scores_df)

['flown_distance', 'flight_duration', 'wtc', 'arrival_time_hour', 'arrival_time_hour_minute', 'actual_offblock_time_hour', 'actual_offblock_time_hour_minute', 'country_code_adep', 'country_code_ades', 'taxiout_time']
Total number of combinations: 1024
